In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import gc
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_numeric_features, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days, date_add_times
from func.ml_utils import save_feature, get_cnt_feature, get_dummie_feature, get_label_feature
from func.parallel_utils import get_parallel_arg_list
from joblib import delayed, Parallel

In [2]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, 'ProductCD']

train_paths = glob('../feature/eda_base/*_train.gz')
test_paths = glob('../feature/eda_base/*_test.gz')

train_paths = [path for path in train_paths 
               if path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count('time_zone')
               or path.count('hour')
               or path.count('Product')
               or path.count('card')
               or path.count('addr')
              ]
test_paths = [path for path in test_paths 
               if path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count('time_zone')
               or path.count('hour')
               or path.count('Product')
               or path.count('card')
               or path.count('addr')
              ]

df_train = parallel_load_data(train_paths)
df_test = parallel_load_data(test_paths)
data = pd.concat([df_train, df_test], axis=0, ignore_index=True)
if COLUMN_ID in data.columns:
    data.set_index(COLUMN_ID, inplace=True)
    
del df_train, df_test
gc.collect()

base_train = read_pkl_gzip('../input/base_train.gz').set_index(COLUMN_ID)
base_test = read_pkl_gzip('../input/base_test.gz').set_index(COLUMN_ID)
length_train = base_train.shape[0]
base = pd.concat([base_train, base_test], axis=0)

In [3]:
START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')

data['datetime'] = data['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
data['datetime'].fillna(datetime.date(2020, 1, 1), inplace=True)
data['date'] = data['datetime'].map(lambda x: x.date())
data['hour'] = data['datetime'].map(lambda x: x.hour)

In [5]:
#========================================================================
# ProductCDあたりのC, V
#========================================================================
cols_cat = [col for col in data.columns if col.count('hour') or col.count('card') or col.count('addr') or col.count('zone')]
cols_pcd = data['ProductCD'].unique()

for col in tqdm(cols_cat):
    for pcd in cols_pcd:
        feature_name = f'{col}__ProductCD-{pcd}'
        data[feature_name] = -1
        data.loc[data['ProductCD'].isin([pcd]), feature_name] = data.loc[data['ProductCD'].isin([pcd]), col]

100%|██████████| 10/10 [00:18<00:00,  2.96s/it]


In [7]:
cols_feature = [col for col in data.columns if col.count('ProductCD-') ]

In [12]:
# User別に期間を切って集計できる様にする
prefix = '604'
dir_save = 'valid'

def parallel_agg(df, feature):
    
    feature_name = f'{feature}_count'
    df[feature_name] = df[feature].map(df[feature].value_counts())

    tmp_train = df.iloc[:length_train]
    tmp_test  = df.iloc[length_train:]
    
    if tmp_train[feature_name].value_counts().shape[0]>2 and tmp_test[feature_name].value_counts().shape[0]>2:
        save_feature(tmp_train[[feature_name]], prefix, dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
        save_feature(tmp_test[ [feature_name]],  prefix, dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)
    
    
list_p = Parallel(60)(
    [delayed(parallel_agg)(
        data[['ProductCD', feature]], feature
    ) for feature in cols_feature])